In [13]:
# IMPORT STATEMENTS

#Import Python packages
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
import ast

# import matplotlib and seaborn to plot charts and graphs
import matplotlib.pyplot as plt
import seaborn as sns

# Import Snowflake modules
from snowflake.snowpark import Session

In [14]:
# Get account credentials from a json file
with open("data_scientist_auth.json") as f:
    data = json.load(f)
    username = data["username"]
    password = data["password"]
    account = data["account"]

# Specify connection parameters
connection_parameters = {
    "account": account,
    "user": username,
    "password": password,
    "role": "TASTY_BI",
    "warehouse": "TASTY_BI_WH",
    "database": "frostbyte_tasty_bytes",
    "schema": "analytics",
}

# Create Snowpark session
session = Session.builder.configs(connection_parameters).create()

In [15]:
# retrieve order details usa table from snowflake
order_data_from_snowflake = session.table("frostbyte_tasty_bytes.analytics.ORDER_DETAILS_USA_MATCHED")

In [16]:
# convert order data to pandas
order_data = order_data_from_snowflake.to_pandas()

In [17]:
# retrieve order details usa table from snowflake
menu_data_from_snowflake = session.table("frostbyte_tasty_bytes.raw_pos.menu")

In [18]:
# convert menu data to pandas
menu_data = menu_data_from_snowflake.to_pandas()

In [19]:
# preview order_data
order_data.head()

,ORDER_ID,CUSTOMER_ID,TRUCK_ID,LOCATION_ID,SHIFT_ID,SHIFT_START_TIME,SHIFT_END_TIME,ORDER_CHANNEL,ORDER_TS,SERVED_TS,...,E_MAIL,PHONE_NUMBER,ORDER_DETAIL_ID,MENU_ITEM_ID,DISCOUNT_ID,LINE_NUMBER,QUANTITY,UNIT_PRICE,PRICE,ORDER_ITEM_DISCOUNT_AMOUNT
0,455595510,140332,31,3003.0,200561976,08:00:00,14:00:00,None,2022-07-12 10:48:16,None,...,Tyree.Hayes@ymail.com,568-629-1049,893970937,18,None,0,1,5.0,5.0,None
1,446723980,104838,68,2043.0,200523048,08:00:00,14:00:00,None,2022-08-06 08:18:18,None,...,Martin.Glass@hotmail.com,750-162-0308,869651471,82,None,0,2,15.0,30.0,None
2,446723980,104838,68,2043.0,200523048,08:00:00,14:00:00,None,2022-08-06 08:18:18,None,...,Martin.Glass@hotmail.com,750-162-0308,869651472,83,None,1,1,9.0,9.0,None
3,446769656,24902,69,3467.0,200523110,08:00:00,14:00:00,None,2022-08-08 08:09:36,None,...,Karley.Dominguez@hotmail.com,030-751-3571,869774770,91,None,0,2,11.0,22.0,None
4,446769656,24902,69,3467.0,200523110,08:00:00,14:00:00,None,2022-08-08 08:09:36,None,...,Karley.Dominguez@hotmail.com,030-751-3571,869774771,93,None,1,2,9.0,18.0,None


In [20]:
# preview menu data
menu_data.head()

,MENU_ID,MENU_TYPE_ID,MENU_TYPE,TRUCK_BRAND_NAME,MENU_ITEM_ID,MENU_ITEM_NAME,ITEM_CATEGORY,ITEM_SUBCATEGORY,COST_OF_GOODS_USD,SALE_PRICE_USD,MENU_ITEM_HEALTH_METRICS_OBJ
0,10001,1,Ice Cream,Freezing Point,10,Lemonade,Beverage,Cold Option,0.65,3.5,"{\n ""menu_item_health_metrics"": [\n {\n ..."
1,10002,1,Ice Cream,Freezing Point,11,Sugar Cone,Dessert,Cold Option,2.50,6.0,"{\n ""menu_item_health_metrics"": [\n {\n ..."
2,10003,1,Ice Cream,Freezing Point,12,Waffle Cone,Dessert,Cold Option,2.50,6.0,"{\n ""menu_item_health_metrics"": [\n {\n ..."
3,10004,1,Ice Cream,Freezing Point,13,Two Scoop Bowl,Dessert,Cold Option,3.00,7.0,"{\n ""menu_item_health_metrics"": [\n {\n ..."
4,10005,1,Ice Cream,Freezing Point,14,Bottled Water,Beverage,Cold Option,0.50,2.0,"{\n ""menu_item_health_metrics"": [\n {\n ..."


In [23]:
order_data = order_data[["MENU_ITEM_ID", "PRICE"]]
order_data.head()

,MENU_ITEM_ID,PRICE
0,18,5.0
1,82,30.0
2,83,9.0
3,91,22.0
4,93,18.0


In [25]:
menu_data = menu_data[["MENU_ITEM_ID", "MENU_TYPE"]]
menu_data.head()

,MENU_ITEM_ID,MENU_TYPE
0,10,Ice Cream
1,11,Ice Cream
2,12,Ice Cream
3,13,Ice Cream
4,14,Ice Cream


In [26]:
# Merge the two DataFrames based on the 'MENU_ITEM_ID' column
merged_data = pd.merge(order_data, menu_data, on='MENU_ITEM_ID')

# Group the data by 'MENU_TYPE' and calculate the total sales (sum of 'PRICE') for each menu type
total_sales_by_menu_type = merged_data.groupby('MENU_TYPE')['PRICE'].sum().reset_index()

total_sales_by_menu_type

,MENU_TYPE,PRICE
0,BBQ,136235.00
1,Chinese,111231.00
2,Crepes,139246.00
3,Ethiopian,123905.00
4,Grilled Cheese,66229.00
5,Gyros,151515.00
6,Hot Dogs,97090.00
7,Ice Cream,60130.50
8,Indian,176570.00
9,Mac & Cheese,123978.00
